<a href="https://colab.research.google.com/github/gabriel-masson/Chika_2025.1/blob/main/Hiperpar%C3%A2metro_fs_paho.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Introduction

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [21]:
!pip install scikit-optimize

In [19]:
!pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 9.0 MB/s eta 0:00:00


In [20]:
import pandas as pd
import numpy as np

from sklearn.neural_network import MLPClassifier

from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier

from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler


from sklearn.model_selection import cross_val_score, LeaveOneOut, KFold

from imblearn.under_sampling import RandomUnderSampler

from sklearn.model_selection import train_test_split

from sklearn.metrics import accuracy_score, roc_auc_score, confusion_matrix, classification_report

from sklearn.model_selection import cross_validate
from sklearn.metrics import make_scorer, f1_score

import matplotlib.pyplot as plt
import seaborn as sns

from skopt import BayesSearchCV
from skopt.space import Real, Categorical, Integer
import catboost

#-

In [5]:
df = pd.read_excel('/content/drive/MyDrive/_MESTRADO/REPLICK/Base de Dados/V2_BANCO_DOTLAB_347.xlsx')
df.head()

,PID,D0_Diagnóstico CHIKV,D0_Método diagnóstico,D0_Data de início dos sintomas,D0_Data de recrutamento,D0_Fase CHIKF (D0),D0_DPIS ambulat-D0,D0_Dias após inicio de sintomas (DPIS)- recrut,D0_Novo critério (0-3/4-10),D0_Critério anterior (0/1-10),...,D0_intensdor_dedomaod,D0_intensdor_quadrile,D0_intensdor_quadrild,D0_intensdor_joelhoe,D0_intensdor_joelhod,D0_intensdor_tornozeloe,D0_intensdor_tornozelod,D0_intensdor_dedopee,D0_intensdor_dedoped,D0_categoria_ativdoenca
0,REC001,Positivo,TR,2019-08-01,2019-09-09,Subagudo,28,39,1,1,...,4,1,1,3,3,4,4,3,2,3
1,REC002,Positivo,TR,2019-09-01,2019-09-16,Subagudo,10,15,1,1,...,4,4,4,4,4,4,4,1,1,2
2,REC004,Positivo,RT-qPCR (ZDC),2020-07-04,2020-07-08,Agudo,4,4,1,1,...,4,1,1,1,1,1,1,1,1,4
3,REC005,Positivo,TR,2020-05-26,2020-07-08,Subagudo,43,43,1,1,...,3,2,2,2,2,4,4,3,3,4
4,REC006,Positivo,TR,2020-06-19,2020-07-13,Subagudo,24,24,1,1,...,2,1,1,1,1,2,2,3,2,3


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 347 entries, 0 to 346
Columns: 252 entries, PID to D0_categoria_ativdoenca
dtypes: datetime64[ns](4), float64(12), int64(227), object(9)
memory usage: 683.3+ KB


##Pré-processamento

In [ ]:
def avaliation_model(y_test,y_pred):
  metricas = pd.DataFrame(classification_report(y_test, y_pred, output_dict=True, zero_division=1)).round(4).T.drop('support', axis=1)
  metricas.drop(['macro avg', 'weighted avg'], axis=0, inplace=True)
  auc = roc_auc_score(y_test, y_pred)

  accuracy= (metricas.loc["accuracy", "f1-score"]*100).round(6)
  recall= (metricas.loc["1", "recall"]*100).round(6)
  especificity= (metricas.loc["0", "recall"]*100).round(6)
  precision= (metricas.loc["1", "precision"]*100).round(6)
  f1_score= (metricas.loc["1", "f1-score"]*100).round(6)
  AUC= (auc*100).round(2)
  print(f'Accuracy: {accuracy}')
  print(f'Recall: {recall}')
  print(f'Rspecificity: {especificity}')
  print(f'Precision: {precision}')
  print(f'F1_score: {f1_score}')
  print(f'AUC: {AUC}')


In [ ]:
def cross_val_models_f1(df_train, model):
    X = df_train.drop('D0_Novo critério (0-3/4-10)', axis=1)
    y = df_train['D0_Novo critério (0-3/4-10)'].astype(int)

    kf = KFold(n_splits=10, shuffle=True, random_state=1)
    f1_scorer = make_scorer(f1_score, average='weighted')

    cross_model = cross_validate(model, X, y, cv=kf, scoring=f1_scorer)
    return cross_model['test_score']

In [6]:
#Features segundo GUIDELINES FOR THE CLINICAL DIAGNOSIS AND TREATMENT OF DENGUE, CHIKUNGUNYA, AND ZIKA By PAHO

features = [ 'D0_idade', 'D0_genero', 'D0_cor',

            'D0_febre_episodio1',         'D0_cefaleia_episodio1',          'D0_fadiga_episodio1',
             'D0_palidez_episodio1',     'D0_hiperemia_episodio1',          'D0_turvacao_episodio1',
             'D0_nausea_episodio1',      'D0_vomito_episodio1',             'D0_diarreia_episodio1',
             'D0_parestesia_episodio1',  'D0_disestesia_episodio1',         'D0_artralgia_episodio1',
             'D0_edema_episodio1',       'D0_mialgia_episodio1',            'D0_lombalgia_episodio1',
             'D0_prurido_episodio1',     'D0_rash_episodio1',               'D0_lesoes_episodio1',
             'D0_alopecia_episodio1',    'D0_alteracaomemoria_episodio1',   'D0_alteracaolibido_episodio1',
             'D0_enpp_fadiga',

             'D0_intensdor_ombroe',
             'D0_intensdor_ombrod',
             'D0_intensdor_punhoe',
             'D0_intensdor_punhod',
             'D0_intensdor_dedomaoe',
             'D0_intensdor_dedomaod',
             'D0_intensdor_joelhoe',
             'D0_intensdor_joelhod',
             'D0_intensdor_tornozeloe',
             'D0_intensdor_tornozelod',
             'D0_intensdor_dedopee',
             'D0_intensdor_dedoped',
             'D0_Novo critério (0-3/4-10)',
]

df_train = df[features]

In [8]:
df_train.fillna(0, inplace=True)

/tmp/ipython-input-8-1491490294.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train.fillna(0, inplace=True)


In [9]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 347 entries, 0 to 346
Data columns (total 38 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   D0_idade                       347 non-null    int64  
 1   D0_genero                      347 non-null    int64  
 2   D0_cor                         347 non-null    int64  
 3   D0_febre_episodio1             347 non-null    int64  
 4   D0_cefaleia_episodio1          347 non-null    int64  
 5   D0_fadiga_episodio1            347 non-null    int64  
 6   D0_palidez_episodio1           347 non-null    int64  
 7   D0_hiperemia_episodio1         347 non-null    int64  
 8   D0_turvacao_episodio1          347 non-null    int64  
 9   D0_nausea_episodio1            347 non-null    int64  
 10  D0_vomito_episodio1            347 non-null    int64  
 11  D0_diarreia_episodio1          347 non-null    int64  
 12  D0_parestesia_episodio1        347 non-null    int

In [11]:
average = 173
X = df_train.drop('D0_Novo critério (0-3/4-10)', axis=1)
y = df_train['D0_Novo critério (0-3/4-10)']

X = df_train.drop('D0_Novo critério (0-3/4-10)', axis=1)

y = df_train['D0_Novo critério (0-3/4-10)'].astype(int)
under_sampler = RandomUnderSampler(sampling_strategy={1: average}, random_state=42)
over_sampler = RandomOverSampler(sampling_strategy={0: average}, random_state=42)
X_resampled, y_resampled = under_sampler.fit_resample(X, y)
X_resampled, y_resampled = over_sampler.fit_resample(X_resampled, y_resampled)

X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.3, random_state=42)

##Bayes

In [26]:
def bayes_search_results(model,name):
  # Obter os melhores parâmetros
  melhor_parametro = model.best_params_

  # Converter o dicionário em DataFrame
  df_melhor_parametro = pd.DataFrame.from_dict(melhor_parametro, orient='index', columns=['Valor'])
  # Adicionar colunas de nome e tipo
  df_melhor_parametro.index.names = ['Nome do Parâmetro']
  df_melhor_parametro['Tipo'] = df_melhor_parametro.index.dtype

  # Exibir o DataFrame
  df_melhor_parametro.to_excel(f'/content/drive/MyDrive/_MESTRADO/REPLICK/Colabs/2025.1/PAHO guideline exemperimentos/BS_results/{name}_melhor_parametro.xlsx')
  return df_melhor_parametro
def bayes_search(X_train, y_train, model, search_space):
  model_bayes = BayesSearchCV(
    model,
    search_space,
    n_iter=1,
    cv=5,
    random_state=42,
    verbose=100
)

  # Realize a busca bayesiana
  model_bayes.fit(X_train, y_train)

    # Os melhores hiperparâmetros encontrados
  print("Melhores hiperparâmetros:", model_bayes.best_params_)

  # A melhor pontuação (geralmente uma métrica de desempenho, como precisão)
  print("Melhor pontuação:", model_bayes.best_score_)

  bayes_search_results(model_bayes, model.__class__.__name__)

In [27]:
models_and_params = {
    'AdaBoost': {
        'model': AdaBoostClassifier(random_state=42),
        'params': {
            'n_estimators': Integer(10, 1000),
            'learning_rate': Real(0.01, 2.0),
            'algorithm': Categorical(['SAMME', 'SAMME.R']),
        },
    },
    'GradientBoosting': {
        'model': GradientBoostingClassifier(),
        'params': {
            'n_estimators': Integer(10, 1000),
            'max_depth': Integer(2, 1000),
            'min_samples_split': Real(0.01, 0.99),
            'min_samples_leaf': Real(0.01, 0.5),
            'criterion': Categorical(['gini', 'entropy']),
        },
    },
    'RandomForest': {
        'model': RandomForestClassifier(random_state=42),
        'params': {
            'n_estimators': Integer(10, 1000),
            'max_depth': Integer(2, 1000),
            'min_samples_split': Real(0.01, 0.99),
            'min_samples_leaf': Real(0.01, 0.5),
            'criterion': Categorical(['gini', 'entropy']),
        },
    },
    'XGBoost': {
        'model': xgb.XGBClassifier(),
        'params': {
            'n_estimators': Integer(10, 999),
            'learning_rate': Real(0.01, 1.0),
            'max_depth': Integer(2, 999),
            'subsample': Real(0.01, 1.0),
            'colsample_bytree': Real(0.01, 0.5),
        },
    },
    'CatBoost': {
        'model': catboost.CatBoostClassifier(verbose=False, random_state=42),
        'params': {
            'iterations': Integer(10, 1000),
            'learning_rate': Real(0.01, 1.0),
            'depth': Integer(2, 1000),
            'l2_leaf_reg': Real(0.01, 10.0),
            'border_count': Integer(10, 254),
            'bagging_temperature': Real(0.0, 10.0),
        },
    },
}


In [28]:
for name, params in models_and_params.items():
  bayes_search(X_train, y_train, params['model'], params['params'])

Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV 1/5; 1/1] START algorithm=SAMME, learning_rate=1.458174228922877, n_estimators=934


/usr/local/lib/python3.11/dist-packages/sklearn/ensemble/_weight_boosting.py:519: FutureWarning: The parameter 'algorithm' is deprecated in 1.6 and has no effect. It will be removed in version 1.8.
  warnings.warn(


[CV 1/5; 1/1] END algorithm=SAMME, learning_rate=1.458174228922877, n_estimators=934;, score=0.633 total time=   2.5s
[CV 2/5; 1/1] START algorithm=SAMME, learning_rate=1.458174228922877, n_estimators=934


/usr/local/lib/python3.11/dist-packages/sklearn/ensemble/_weight_boosting.py:519: FutureWarning: The parameter 'algorithm' is deprecated in 1.6 and has no effect. It will be removed in version 1.8.
  warnings.warn(


[CV 2/5; 1/1] END algorithm=SAMME, learning_rate=1.458174228922877, n_estimators=934;, score=0.653 total time=   2.0s
[CV 3/5; 1/1] START algorithm=SAMME, learning_rate=1.458174228922877, n_estimators=934


...

[CV 4/5; 1/1] END algorithm=SAMME, learning_rate=1.458174228922877, n_estimators=934;, score=0.667 total time=   3.0s
[CV 5/5; 1/1] START algorithm=SAMME, learning_rate=1.458174228922877, n_estimators=934


/usr/local/lib/python3.11/dist-packages/sklearn/ensemble/_weight_boosting.py:519: FutureWarning: The parameter 'algorithm' is deprecated in 1.6 and has no effect. It will be removed in version 1.8.
  warnings.warn(


[CV 5/5; 1/1] END algorithm=SAMME, learning_rate=1.458174228922877, n_estimators=934;, score=0.688 total time=   2.4s


/usr/local/lib/python3.11/dist-packages/sklearn/ensemble/_weight_boosting.py:519: FutureWarning: The parameter 'algorithm' is deprecated in 1.6 and has no effect. It will be removed in version 1.8.
  warnings.warn(


Melhores hiperparâmetros: OrderedDict([('algorithm', 'SAMME'), ('learning_rate', 1.458174228922877), ('n_estimators', 934)])
Melhor pontuação: 0.6904761904761905
